<!--
Pavel Prochazka 
pavel@prochazka.info
v0.2.0 21.12.2016
-->

#Numerical Simulation

This section aims with the implementation of the system according to the network information flow introduced in the first section with constellation design shown in the second section and demodulation technique shown in the third section.

##Gaussian Noise

Starting with Gaussian noise description, the quality of all links are described by signal to noise ratio (SNR or $\gamma$) that is unambiguously related to variance of the Gaussian noise ($\sigma_W^2$), since unit power restriction is assumed in all transmiting nodes. The relation
$$\sigma_W^2 = 10^{\frac{\gamma}{10}}$$ is described as:

In [8]:
def SNR2sigma2w(SNR):
    alpha = 10**(float(SNR)/10)
    return 1.0/alpha

The complex valued zero mean AWGN is then generated according to its pdf $$p(w)=\frac{1}{\sigma_W\pi}\exp\left(-\frac{|w|^2}{\sigma_W^2}\right)$$ as:

In [9]:
import numpy as np
SNR = 8 # Signal to noise ratio
L = 1000 # Vector length
sigma2w = SNR2sigma2w(SNR)
# Samples of complex valued Gaussian noise
w = np.sqrt(sigma2w/2) * (np.random.randn(L) + 1j * np.random.randn(L))

One can ensure that the properties of the generated noise fit the assumptions (the mean should tend to zero and mean energy to $\sigma_W^2$ with increasing $L$:

In [13]:
print 'Empirical mean:({0.real:.2f} + {0.imag:.2f}i)'.format(1./L * np.sum(w))
print 'Empirical mean energy:%.3f,'%(1./L * np.sum(np.abs(w)**2))
print 'Sigma2w:%.3f'%sigma2w

Empirical mean:(-0.01 + 0.00i)
Empirical mean energy:0.162,
Sigma2w:0.158


## Constellation Design in Use

When everything is prepared, we can start with the implementation:

It should be stressed that the constellations depend on global state knowledge.

In [50]:
# Definitions
Nb = 1 # Nb bits can be reliably passed through the side channels
Ns = 1 # Ns bits must be passed fully through the relay
Aq_b = 2**Nb # Cardinality of alphabet in sources
Aq_s = 2**Ns # Cardinality of alphabet in sources
L = 1000

gMAC = 18 # SNR of sources to relay channel  [dB]
gBC = 18 # SNR of relay dastinations channel [dB]
gHSI = 20 # SNR of sources to complementary destinations (site) channel  [dB]

# Channel gains (due to symetry symmetry, only destination A
# is considered)
hAR = 1. # sA to relay
hBR = 1. # sB to relay
hRA = 1. # relay to dA
hBA = 1. # sB to dA

# Evaluation of Gaussian variances
sigma2wMAC = SNR2sigma2w(gMAC)
sigma2wBC = SNR2sigma2w(gBC)
sigma2wHSI = SNR2sigma2w(gHSI)

# Evaluation of proper constellations
from Constellation_Design_lib import const_design_XOR
(sA_const, sB_const, basic_part, superposed_part, relay_const, alpha) = \
            const_design_XOR(Nb, Ns, hAR/hBR)

In [42]:
# Source data
dAb = np.random.randint(Aq_b, size=L)  # basic part of source A data
dAs = np.random.randint(Aq_s, size=L)  # superposed part of source A data
dBb = np.random.randint(Aq_b, size=L)  # basic part of source B data
dBs = np.random.randint(Aq_s, size=L)  # superposed part of source B data
dA = (dAs, dAb) # source A data
dB = (dBs, dBb) # source B data

# Constellation mappers
sAb = basic_part[dAb]
sBb = basic_part[dBb]
sAs = superposed_part[dAs]
sBs = 1j*superposed_part[dBs]
sA = sAs + sAb # Modulated data sA
sB = sBs + sBb # Modulated data sB
#import matplotlib.pyplot as plt
#plt.plot(np.real(sB), np.imag(sB), 'x')
#plt.show()

The transmission within the links in the MAC stage can are evaluated as:

In [47]:
# Sources to relay channel
wR = np.sqrt(sigma2wMAC/2) * (np.random.randn(L) + 1j * np.random.randn(L))
xR = hAR * sA + hBR * sB + wR 

# Site Link 
wDA_M = np.sqrt(sigma2wHSI/2) * (np.random.randn(L) + 1j * np.random.randn(L))
zA = hBA * sB + wDA_M

The destination stores the observation for later processing. The relay provides first demodulation to decode the network data that are then modulated to relay symbols. 

In [46]:
# Demodulator in relay
mudAs = np.zeros([L, Aq_s], float) # metric p(x|dAs)
mudBs = np.zeros([L, Aq_s], float) # metric p(x|dBs)
mudb = np.zeros([L, Aq_b], float) # metric p(x|dAb^dBb)

for iAb in range(Aq_b):
    for iAs in range(Aq_s):
        for iBb in range(Aq_b):
            for iBs in range(Aq_s):    
                # Basic part index
                ind_b = iAb ^ iBb
                # likelihood p(x|dA,dB)
                m = np.exp(-np.abs(xR - hAR * (basic_part[iAb] + superposed_part[iAs])\
                                - hBR * (basic_part[iBb] + 1j*superposed_part[iBs]))**2/sigma2wMAC)
                # Marginalization - uniform data distribution assumed                
                mudb[:, ind_b] += m 
                mudAs[:, iAs] += m
                mudBs[:, iBs] += m
est_dAs = np.argmax(mudAs, axis = 1) # decision
est_dBs = np.argmax(mudBs, axis = 1) # decision
est_db = np.argmax(mudb, axis = 1) # decision

dR = (est_dAs, est_dBs, est_db)

print 'nerr in relay: dAs:%d, dBs:%d, db:%d'\
%(np.sum(est_dAs!=dAs), np.sum(est_dBs!=dBs), np.sum(est_db!=(dAb^dBb)))

from Constellation_Design_lib import QAM
rconst = QAM(2*Ns + Nb)[0] # Signal space mapping in Relay
sR = rconst[est_dAs*(Aq_b+Aq_s) + est_dBs*Aq_b + est_db]

nerr in relay: dAs:0, dBs:0, db:0


In [52]:
# BC stage (Relay to dA link)
wDA_B = np.sqrt(sigma2wBC/2) * (np.random.randn(L) + 1j * np.random.randn(L))
yA = hRA * sR + wDA_B # Destination A observation

As it was already said, the relay first process the observation from BC stage to recover both superposed streams and the network function of the basic streams. The complementary superposed stream is used for interference cancellation to improve the site link quality. The complementary basic part is then decoded and the inteded data are then decoded as in the network information flow section.

In [ ]:
# Demodulator Destination  A -- link from relay
mudAs = np.zeros([L, Aq_s], float) # metric p(x|dAs)
mudBs = np.zeros([L, Aq_s], float) # metric p(x|dBs)
mudb = np.zeros([L, Aq_b], float) # metric p(x|dAb^dBb)

for ib in range(Aq_b):
    for iAs in range(Aq_s):        
        for iBs in range(Aq_s):    
            ind = iAs*(Aq_b+Aq_s) + iBs*Aq_b + ib
            m = np.exp(-np.abs(yA - hRA * rconst[ind])**2/sigma2wBC)
            # Marginalization - uniform data distribution assumed                
            mudb[:, ib] += m 
            mudAs[:, iAs] += m
            mudBs[:, iBs] += m
est_dAs = np.argmax(mudAs, axis = 1) # decision
est_dBs = np.argmax(mudBs, axis = 1) # decision
est_db = np.argmax(mudb, axis = 1) # decision

# Interference cancellation from est_dBs
mudBb = np.zeros([L, Aq_b], float) # metric p(x|dBb)
for iBb in range(Aq_b):
    for iBs in range(Aq_b):
    

print 'nerr in relay: dAs:%d, dBs:%d, db:%d'\
%(np.sum(est_dAs!=dAs), np.sum(est_dBs!=dBs), np.sum(est_db!=(dAb^dBb)))
